## BIG G EXPRESS

In [13]:
import pandas as pd
import numpy as np

In [25]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
diagnostics = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
diagnostics.head()

C:\Users\irela\AppData\Local\Temp\ipykernel_25824\2194169026.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
faults_and_diagnostics = faults.merge(diagnostics, how='left', left_on='RecordID', right_on='FaultId')
faults_and_diagnostics.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
faults_and_diagnostics['FullDerate'] = faults_and_diagnostics['spn'].apply(lambda x: 1 if x == 5246 else 0)
faults_and_diagnostics = faults_and_diagnostics.sort_values(['EquipmentID', 'LocationTimeStamp'])

faults_and_diagnostics['EventTimeStamp'] = pd.to_datetime(faults_and_diagnostics['EventTimeStamp'])

faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.where(faults_and_diagnostics['FullDerate']==1)['EventTimeStamp']
faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')

faults_and_diagnostics['HoursUntilNextDerate'] = (faults_and_diagnostics['NextDerateTime'] - faults_and_diagnostics['EventTimeStamp']).dt.total_seconds()/3600.0
faults_and_diagnostics['DerateInNextTwoHours'] = np.where(faults_and_diagnostics['HoursUntilNextDerate'] <= 2, 1, 0)
faults_and_diagnostics.head()

C:\Users\irela\AppData\Local\Temp\ipykernel_25824\1602781329.py:7: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')
C:\Users\irela\AppData\Local\Temp\ipykernel_25824\1602781329.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  faults_and_diagnostics['NextDerateTime'] = faults_and_diagnostics.groupby('EquipmentID')['NextDerateTime'].fillna(method='bfill')


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,FullDerate,NextDerateTime,HoursUntilNextDerate,DerateInNextTwoHours
33355,34467,2092800,2015-04-26 06:16:16,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10612.616667,0
35080,36192,2123154,2015-04-28 05:29:21,High (Severity High) Engine Speed,NaN,unknown,unknown,unknown,unknown,49,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10565.398611,0
47186,48298,2346346,2015-05-10 07:11:34,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.728246,3276.75,8.8,0.87,0,2016-07-11 10:53:16,10275.695000,0
47213,48325,2346621,2015-05-10 07:59:25,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2016-07-11 10:53:16,10274.897500,0
48303,49415,2363162,2015-05-11 13:11:20,Incorrect Data J1939 Network #1 Primary Vehicl...,NaN,unknown,unknown,unknown,unknown,11,...,False,NaN,4.475814,3276.75,14.4,0.58,0,2016-07-11 10:53:16,10245.698889,0


In [30]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error

In [ ]:
faults_and_diagnostics = faults_and_diagnostics[faults_and_diagnostics['spn'] != 5246]

X = faults_and_diagnostics.drop(columns=['FullDerate', 'NextDerateTime', 'HoursUntilNextDerate', 'DerateInNextTwoHours'])
y = faults_and_diagnostics['HoursUntilNextDerate']

cat_cols = X.select_dtypes(include='object').columns
X[cat_cols] = X[cat_cols].astype(str)

# Event indicator: 1 if derate eventually occurred, 0 if it was censored (no known future derate)
event_indicator = (faults_and_diagnostics['HoursUntilNextDerate'].notna()).astype(int)

# Now create 2D y array
y_survival = np.vstack([faults_and_diagnostics['HoursUntilNextDerate'].fillna(9999), event_indicator]).T

X_train, X_test, y_train, y_test = train_test_split(X, y_survival, random_state=321)


numeric_selector = make_column_selector(dtype_include='number')
categorical_selector = make_column_selector(dtype_include='object')

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer([
    ('num', numeric_transformer, numeric_selector),
    ('cat', categorical_transformer, categorical_selector)
])

pipe = Pipeline([
    ('ct', ct),
    ('model', XGBRegressor(objective='reg:squarederror'))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print("RMSE:", mean_squared_error(y_test[:, 0], y_pred, squared=False))

c:\Users\irela\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\irela\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\irela\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
#import xgboost
#xgboost.XGBRegressor(objective='survival:cox')